> FIAP - Redes Neurais Artificiais, Deep Learning e Algoritmos Genéticos

# Perceptron - Está na hora de dormir?

Aqui vamos criar um Perceptron simples que define se está na hora de dormir ou não.

## Definição das funções básicas

Vamos criar as funções básicas de soma $\sum_{i=1}^m x_i w_i = u$ e da função de ativação degrau $f(u + b)$


In [86]:
# TODO: Colocar isso em uma function chamada Perceptron ou uma classe

from functools import reduce


class Perceptron:

    # TODO: Essa função precisa logar / retornar os valores finais para nós conseguirmos debuggar.
    # Precisamos ver como a cada relação entre valor -> peso está funcionando para ajustarmos nosso viés e pesos.
    # Acho melhor adicionar uma função nova ao invés de mexer nessa?

    # Soma ponderada

    def sum_data(data: tuple[dict[str, float]]):
        return reduce(lambda total, current: total + (current['value'] * current['weight']), data, 0)

    # Função degrau

    def threshold_function(total_sum: float, bias: float):
        threshold = total_sum + bias

        if threshold < 0:
            return 0

        return 1

    def show_detailed_sum(data: tuple[dict[str, float]]):
        output = []

        for item in data:
            output.append({
                'column_name': item['column_name'],  # for debug
                'value': item['value'],
                'weight': item['weight'],
                'sum': item['weight'] * item['value'],
            })

        return output


# Tests

total_sum = Perceptron.sum_data([
    {'value': 1, 'weight': 0.10},
    {'value': 10, 'weight': 0.05},
])

assert total_sum == 0.6, "The sum should equal 0.6"


# Tests

assert Perceptron.threshold_function(
    0.6, bias=0) == 1, "Should be 1 = neuron ON"

assert Perceptron.threshold_function(
    0.2, bias=-1) == 0, "Should be 0 = neuron OFF"


## Apresentação do problema

Temos o seguinte cenário: Precisamos saber se está na hora de dormir baseado nos dados coletados de uma pessoa ao longo do tempo.

### Base de dados

Nossa base de dados vem do arquivo `sleep_dataset.csv` com os as variáveis relacionadas ao sono da pessoa. As variáveis são:

| Nome                        | Descrição                                                                                                                                 | Possíveis valores                                                                                                                                  | Peso  |
| --------------------------- | ----------------------------------------------------------------------------------------------------------------------------------------- | -------------------------------------------------------------------------------------------------------------------------------------------------- | ----- |
| prev_sleep_hours            | A quantidade de horas de sono que a pessoa teve no último sono                                                                            | De 0 a 24                                                                                                                                          | -0.02 |
| active_minutes              | A quantidade de minutos desde o último sono em que ele passou fazendo atividades físicas e fazendo raciocínios lógicos (atividade mental) | De 30 a 400                                                                                                                                        | 0.005 |
| awake_since_hours           | A quantidade de horas desde a última vez que a pessoa dormiu                                                                              | De 0 a 36                                                                                                                                          | 0.1   |
| comfortable_sleep_available | A pessoa tem acesso a uma posição confortável para dormir?                                                                                | 1 = Sim; 0 = Não                                                                                                                                   | 0.1   |
| mental_status               | Status mental da pessoa no momento                                                                                                        | 0 = Normal / Tranquila; 1 = Triste; 2 = Ansiosa / Nervosa                                                                                          | -0.25 |
| physical_status             | Status físico do corpo da pessoa no momento                                                                                               | -1 = Cansada; 0 = Normal / Sem dores; 1 = Desconforto; 2 = Dores simples ou fome/sede; 3 = Dores severas                                           | -0.25 |
| activity                    | Atividade atual da pessoa                                                                                                                 | -1 = Atividades passivas; 0 = Sem atividade; 1 = Atividades simples; 2 = Atividades complexas; 3 = Concentração total                              | -0.15 |
| visual_stimulation          | Estimulação visual atual na pessoa                                                                                                        | 0 = Sem estimulação; 1 = Estimulação baixa (luz baixa e indireta); 2 = Estimulação moderada (luz baixa); 3 = Estimulação alta (luz forte e direta) | -0.25 |
| audibly_stimulation         | Estimulação sonora atual na pessoa                                                                                                        | 0 = Barulhos leves ou música leve; 1 = Barulhos leve e sincronizado; 2 = Barulho moderado; 3 = Barulho alto                                        | -0.25 |
| posture                     | Posição / postura atual da pessoa                                                                                                         | -3 = Correndo; -2 Andando; -1 Em pé; 0 = Sentado; 1 = Deitado;                                                                                     | 0.45  |


### Desmonstração dos dados

Uma demonstração dos valores da nossa base de dados

In [87]:
import pandas as pd

df = pd.read_csv('data/sleep_dataset.csv', index_col="ID")

df.head()


,prev_sleep_hours,active_minutes,awake_since_hours,comfortable_sleep_available,mental_status,physical_status,activity,visual_stimulation,audibly_stimulation,posture
ID,,,,,,,,,,
1,8,60,15,1,0,-1,1,1,2,1
999,6,120,12,1,0,0,1,2,0,-1
882,9,100,10,1,0,-1,0,1,0,0
92,9,30,4,1,0,1,1,2,2,0
79,8,120,8,1,0,0,1,1,0,0


### Definição dos pesos no código

Vamos criar a varíavel `weights` que vai servir para definir todos os pesos das nossas variáveis da base de dados

In [88]:
weights = {
    "prev_sleep_hours": -0.02,
    "active_minutes": 0.005,
    "awake_since_hours": 0.1,
    "comfortable_sleep_available": 0.1,
    "mental_status": -0.25,
    "physical_status": -0.25,
    "activity": -0.15,
    "visual_stimulation": -0.25,
    "audibly_stimulation": -0.25,
    "posture": 0.45,
}

## Executando nosso perceptron

### Formatação dos dados (inclusão de peso)

Vamos fazer a inclusão dos pesos da nossa base de dados e formatar como necessário pela função `sum_data`

In [89]:
data = []

for row in df.iloc:
    values_with_weight = []

    for column_name, value in row.to_dict().items():
        values_with_weight.append({
            'column_name': column_name,  # for debug
            'value': value,
            'weight': weights[column_name],

        })

    data.append({'id': row.name, 'data': values_with_weight})

Agora temos os seguintes dados na variável `data`:

In [90]:
pd.DataFrame(data[0]['data'])

,column_name,value,weight
0,prev_sleep_hours,8,-0.020
1,active_minutes,60,0.005
2,awake_since_hours,15,0.100
3,comfortable_sleep_available,1,0.100
4,mental_status,0,-0.250
5,physical_status,-1,-0.250
6,activity,1,-0.150
7,visual_stimulation,1,-0.250
8,audibly_stimulation,2,-0.250
9,posture,1,0.450


Vamos analisar os nossos valores e pesos:

In [91]:
sum_details = []

for row in data:
  sum_details.append(Perceptron.show_detailed_sum(row['data']))

pd.DataFrame(sum_details[1])

,column_name,value,weight,sum
0,prev_sleep_hours,6,-0.020,-0.12
1,active_minutes,120,0.005,0.60
2,awake_since_hours,12,0.100,1.20
3,comfortable_sleep_available,1,0.100,0.10
4,mental_status,0,-0.250,-0.00
5,physical_status,0,-0.250,-0.00
6,activity,1,-0.150,-0.15
7,visual_stimulation,2,-0.250,-0.50
8,audibly_stimulation,0,-0.250,-0.00
9,posture,-1,0.450,-0.45


Com isso, podemos executar nosso Perceptron:



In [92]:
result = []
bias = -1

for row in data:
    total_row_sum = Perceptron.sum_data(row['data'])

    neuron_activation = Perceptron.threshold_function(total_row_sum, bias)

    result.append({
        'id': row['id'],
        'Soma total': total_row_sum,
        'bias': bias,
        'f(x)': total_row_sum + bias,
        'Neurônio ativo?': neuron_activation
    })

result_df = pd.DataFrame(result).set_index('id')

result_df

,Soma total,bias,f(x),Neurônio ativo?
id,,,,
1,1.54,-1,0.54,1
999,0.68,-1,-0.32,0
882,1.42,-1,0.42,1
92,-0.93,-1,-1.93,0
79,0.94,-1,-0.06,0
